# LDA classification of CyTOF

#### Verifying results of Predicting   cell   types   in   single   cell   mass   cytometry data

In [14]:
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import numpy as np
from sklearn.metrics import f1_score
import os
from sklearn.model_selection import KFold
from sklearn import metrics
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


### Sample of data
28 features over a large number of cells per CSV

In [15]:
sample = pd.read_csv("../Data/MultiCenter/Samples/sample01.csv", header=None)
markers = ['CCR6','CD20','CD45','CD14','CD16','CD8','CD3','CD4']
sample.columns = markers
sample.head(10)

,CCR6,CD20,CD45,CD14,CD16,CD8,CD3,CD4
0,17.890000,10.2310,790.55,2.7417,0.00000,1.065300,179.600000,45.0630
1,0.000000,4.6372,1112.70,468.4100,0.25689,2.436200,1.781100,13.5060
2,0.002912,7.2418,1294.20,624.2500,0.00000,0.493150,2.351400,2.0904
3,2.911800,2.7268,1321.20,579.8200,0.00000,0.910060,1.575700,8.3613
4,0.000000,4.5986,778.36,0.0000,0.00000,0.000000,253.720000,38.5670
5,501.770000,921.6700,934.45,2.6003,0.00000,0.043438,2.614300,1.7870
6,0.753110,2.8833,964.34,383.8900,4.11720,0.000000,3.513200,1.4420
7,0.157420,0.0000,748.81,80.8760,1.59330,4.550900,4.110600,5.1319
8,0.000000,1.7064,965.98,1.4520,65.13400,0.186930,0.008875,0.0000
9,0.000000,2.5197,1308.40,0.2186,16.53700,83.937000,4.547600,1.5487


X-fold validation with one file/sample being left out each iteration and used as test:
CV-samples in paper

In [16]:
labelfiles = os.listdir("../Data/MultiCenter/Labels/")
samplefiles = os.listdir("../Data/MultiCenter/Samples/")
labelfiles.sort()
samplefiles.sort()



## Stratify MultiCenter per sample
Train 10 iterations of MultiCenter with 9 samples, leaving one out <br>
Predict labels for sample which was excluded from training <br>
Save predictions to file and prediction accuracy to variable <br>
MultiCenter CV-samples in paper

In [17]:
kfoldsplitter = KFold(n_splits=len(samplefiles))
for train, test in kfoldsplitter.split(labelfiles):
    classifier = LinearDiscriminantAnalysis()
    for trainingset in train:
        X = pd.read_csv("../Data/MultiCenter/Samples/" + samplefiles[trainingset], header=None).values #train
        y = pd.read_csv("../Data/MultiCenter/Labels/" + labelfiles[trainingset], header=None).values.ravel() #labels
        classifier.fit(X, y)
    testdata = pd.read_csv("../Data/MultiCenter/Samples/" + samplefiles[test[0]], header=None).values #test
    testlabels = pd.read_csv("../Data/MultiCenter/Labels/" + labelfiles[test[0]], header=None).values.ravel() #testlabels
    #score testdata with its true labels in model trained against 9 other samples & save to file
    prediction = classifier.predict(testdata)
    with open('../Results/Predictions/MultiCenter LDA/CV-sample/predict_'+samplefiles[test[0]], 'w') as f:
        for item in prediction:
            f.write("%s\n" % item)
    print("Done testing %s" % samplefiles[test[0]])


Done testing sample01.csv
Done testing sample02.csv
Done testing sample03.csv
Done testing sample04.csv
Done testing sample05.csv
Done testing sample06.csv
Done testing sample07.csv
Done testing sample08.csv
Done testing sample09.csv
Done testing sample10.csv
Done testing sample11.csv
Done testing sample12.csv
Done testing sample13.csv
Done testing sample14.csv
Done testing sample15.csv
Done testing sample16.csv


## Stratify PANORAMA 5-fold

In [18]:
kfoldsplitter = KFold(n_splits=5)

#Load all samples and labels for 5-cold CV
samplesheap = pd.DataFrame()
labelsheap = pd.DataFrame()

for labelname, samplename in zip(labelfiles, samplefiles):
    samplesheap = samplesheap.append(pd.read_csv("../Data/MultiCenter/Samples/" + samplename, header=None))
    labelsheap = labelsheap.append(pd.read_csv("../Data/MultiCenter/Labels/" + labelname, header=None))



Number of predictions/trues matches original numer of instances

In [19]:


batch = 1
for train, test in kfoldsplitter.split(samplesheap):
    classifier = LinearDiscriminantAnalysis()
    #select test/train/true labels
    testdata = samplesheap.iloc[test].values
    traindata = samplesheap.iloc[train].values
    trainlabels = labelsheap.iloc[train].values.ravel()
    #fit & predict
    classifier.fit(traindata, trainlabels)
    prediction = classifier.predict(testdata)
    #write prediction to file
    print(len(prediction))

    with open('../Results/Predictions/MultiCenter LDA/CV-cells/predict_batch_' + str(batch) +'.csv', 'w') as f:
        for item in prediction:
            f.write("%s\n" % item)
    print("Done testing %s" % str(batch))
    batch += 1

185937
Done testing 1
185937
Done testing 2
185937
Done testing 3
185937
Done testing 4
185937
Done testing 5


In [20]:
#remove enormous datasets from memory
print(len(samplesheap))

del labelsheap
del samplesheap

929685
